In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
# .env 파일 로드 확인
load_dotenv('.env', verbose=True)

hf_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

login(hf_token)


In [4]:
import faiss

from sentence_transformers import SentenceTransformer
import json

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 책 데이터 로드
with open("All_books_with_summary.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 요약을 임베딩 변환
book_summaries = [book["description"] for book in books_data]
book_titles = [book["title"] for book in books_data]

# 🔹 임베딩 변환
book_embeddings = embedding_model.encode(book_summaries, convert_to_numpy=True)

# 🔹 L2 정규화 (코사인 유사도 적용)
faiss.normalize_L2(book_embeddings)  # 벡터 정규화

# 🔹 FAISS 인덱스 생성 (L2 정규화된 벡터 기반)
dimension = book_embeddings.shape[1]  # 벡터 차원 (768)
faiss_index = faiss.IndexFlatIP(dimension)  # 🚀 코사인 유사도 적용 (Inner Product)
faiss_index.add(book_embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(faiss_index, "book_faiss_cosine_index.bin")

print("✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.")


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.


In [5]:
# 🔹 FAISS 인덱스에 저장된 벡터 개수 확인
indexed_vectors_count = faiss_index.ntotal

# ✅ 책 데이터의 개수와 비교
book_count = len(book_summaries)

# 결과 출력
print(f"책 데이터의 개수: {book_count}")
print(f"FAISS 인덱스에 저장된 벡터의 개수: {indexed_vectors_count}")

# ✅ 비교 결과 확인
if book_count == indexed_vectors_count:
    print("✅ 책 데이터와 FAISS 인덱스의 개수가 일치합니다.")
else:
    print("⚠️ 책 데이터와 FAISS 인덱스의 개수가 일치하지 않습니다.")


책 데이터의 개수: 5532
FAISS 인덱스에 저장된 벡터의 개수: 5532
✅ 책 데이터와 FAISS 인덱스의 개수가 일치합니다.


In [8]:
import faiss
import json
from sentence_transformers import SentenceTransformer

# ✅ FAISS 인덱스 로드
faiss_index = faiss.read_index("book_faiss_cosine_index.bin")

# ✅ 책 데이터 로드
with open("All_books_with_summary.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

book_titles = [book["title"] for book in books_data]
book_summaries = [book["description"] for book in books_data]

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")


# ✅ FAISS 검색 함수
def find_similar_books(user_story, top_k=10):
    """유저 입력 스토리 → 코사인 유사도 기반 책 검색"""
    # 🔹 유저 입력을 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)
    faiss.normalize_L2(user_embedding)  # 벡터 정규화

    # 🔹 FAISS 검색
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]
        title = book_titles[idx]
        summary = book_summaries[idx]
        recommended_books.append(f"📖 **{title}**\n📌 {summary}\n")

    return recommended_books
# ✅ 테스트 실행
user_story = "한 소년이 마법사가 되는 이야기"


print(find_similar_books(user_story, top_k=10))


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


['📖 **해리포터와 마법사의 돌 제1권 1**\n📌 꼬마 마법사의 신나는 모험을 그린 베스트셀러 소설. 무서운 이모부부와 함께 계단 밑 작은 방에서 비참한 삶을 살던 해리는 부엉이 한 마리가 배달한 초대장에 인생이 바뀌기 시작한다. 새로운 친구와 온갖 마법을 배우며 해리는 어둠의 힘과 결투를 벌이는데....\n', '📖 **해리포터(Harry Potter): 혼혈 왕자 1**\n📌 선과 악의 대립 속에서 평범한 어린 소년이 한 사람의 영웅으로 성장해나가는 보편적인 테마를 바탕으로 빈틈없는 소설적 구성과 생생하게 살아 있는 캐릭터, 정교하게 만들어낸 환상의 세계를 접목시킨 21세기의 고전 「해리포터 시리즈」의 제6편『해리포터와 혼혈 왕자』 제1부. 볼드모트의 영향력이 커지는 가운데 마법사 세계와 머글 세계는 경계 상태에 들어가게 된다. 덤블도어와 해리의 설득으로 호그와트 교수로 취임한 슬러그혼은 마법약 과목을 가르치고, 스네이프는 어둠의 마법 방어술을 가르치게 된다. 슬러그혼의 첫 수업에서 해리는 혼혈 왕자라는 별칭을 가진 소년이 쓰던 책을 받게 되고, 그 책의 도움으로 마법의 약 수업에서 두각을 보인다.\n', '📖 **해리포터(Harry Potter): 죽음의 성물 2**\n📌 선과 악의 대립 속에서 평범한 어린 소년이 한 사람의 영웅으로 성장해나가는 보편적인 테마를 바탕으로 빈틈없는 소설적 구성과 생생하게 살아 있는 캐릭터, 정교하게 만들어낸 환상의 세계를 접목시킨 21세기의 고전 「해리포터 시리즈」의 제7편『해리포터와 죽음의 성물』 제2부. 열일곱 살이 되기 직전, 더즐리 가에 걸린 보호마법이 걷히기 전에 친구들의 미끼작전으로 은신처로 몸을 피한 해리는 론과 헤르미온느와 함께 덤블도어의 뜻을 이어 호크룩스를 찾기로 결심한다. 덤블도어의 유품을 조사하던 중 죽음을 피하는 강력한 마법 물품인 ‘죽음의 성물’이 실제로 존재하며 그중 하나인 딱총나무 지팡이는 볼드모트가, 나머지 하나인 투명 망토는 자신이 갖고 있다는 것을 알게 된다.\n', '📖 **해리포터(